In [ ]:
'''
Insieme di funzioni richiamate negli script
che compongono il progetto
'''

In [ ]:
# Import delle librerie
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as T
from torchvision.models import resnet50
from torch.utils.data import DataLoader,SubsetRandomSampler

import cv2
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
def fix_seed(seed_value):
    """Inizializzazione generatore valori random"""
    torch.manual_seed(seed_value)    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [ ]:
def train_one_epoch(model, device ,dataloader, loss_func, optimizer, scheduler):
    """Esegue un epoca di training della rete"""
    train_loss , train_correct = 0.0, 0
    model.train()
    for images, labels in dataloader:
        images,labels = images.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float() 
        optimizer.zero_grad()
        output = model(images)
        loss = loss_func(output,labels)
        loss.backward()
        optimizer.step()
        if scheduler: scheduler.step()
        train_loss += loss.item() * images.size(0)
        probs = torch.sigmoid(output)
        preds = probs.round()
        train_correct += (preds == labels).sum().item()
    return train_loss, train_correct

In [ ]:
def test_one_epoch(model, device, dataloader, loss_func):
    """Esegue la validazione durante la K-Fold C.V"""
    test_loss , test_correct = 0.0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:
            images,labels = images.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            output = model(images)
            loss = loss_func(output,labels)
            test_loss += loss.item() * images.size(0)
            probs = torch.sigmoid(output)
            preds = probs.round()  
            test_correct += (preds == labels).sum().item()
        return test_loss, test_correct

In [ ]:
def get_test_preds(model, device, dataloader, loss_func):
    """Esegue la valutazione su un test-set esterno"""
    predictions, targets = [], []
    test_loss , test_correct = 0.0, 0
    model.eval()
    with torch.no_grad():
        for images, labels in dataloader:
            images,labels = images.to(device), labels.to(device)
            labels = labels.unsqueeze(1).float()
            output = model(images)
            loss = loss_func(output,labels)
            test_loss += loss.item() * images.size(0)
            probs = torch.sigmoid(output)
            preds = probs.round()  
            test_correct += (preds == labels).sum().item()
            
            for i in range(len(preds)):
                predictions.append(preds[i].cpu().data.numpy())
                targets.append(labels[i].cpu().data.numpy())
            
        return test_loss, test_correct, predictions, targets

In [ ]:
def reset_wgts(model):
    """Esegue il reset dei pesi della rete durante K-Fold C.V."""
    for _, module in model.named_children():
        if hasattr(module, 'reset_parameters'):
            module.reset_parameters()

In [ ]:
base_path = '/media/users/cgambina/Progetto_6/Repo/FaceDetect-master/'
haarcascade_path = base_path + 'haarcascade_frontalface_default.xml'  
face_cascade = cv2.CascadeClassifier(haarcascade_path)

def get_face(image_path, box_color=(0,0,255)):
    """Individua un volto umano presente in un immagine"""
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
    face = face_cascade.detectMultiScale(image, 1.05, 1)
    
    area = -1
    for (x, y, w, h) in face:
        # Selezione della b-box più grande
        new_area = w * h
        if new_area > area:
            area = new_area
            dims = (x, y, w, h)
               
    cv2.rectangle(image, (dims[0], dims[1]), (dims[0]+dims[2], dims[1]+dims[3]), box_color, 3)
    face = image[y:y + h, x:x + w]
        
    face = cv2.resize(face, (128, 128))
    pil_image = Image.fromarray(face)
    
    plt.imshow(image)    
    return pil_image

In [ ]:
def get_prediction(pil_image, tfms, net):
    """Calcola la probabilità che una persona non indossi una maschera"""
    tensor_img = tfms(pil_image)
    tensor_img = torch.unsqueeze(tensor_img, 0)
    output = net(tensor_img)
    output = torch.sigmoid(output)
    return output.item()

In [ ]:
def freeze_layers(net, layers_list=['layer1', 'layer2']):
    """Inibisce il gradiente nei layer selezionati"""
    layer_counter = 0
    for (name, module) in net.named_children():
        if name in layers_list:
            for layer in module.children():
                for param in layer.parameters():
                    param.requires_grad = False
                    
            print(f'Modulo {name} gradiente inibito!!')
            layer_counter += 1

In [ ]:
def show_batch(data, n_elems=(4,6)):
    """Mostra un campione di immagini"""
    labels_map = ['Con Maschera', 'Senza Maschera']

    figure = plt.figure(figsize=(10, 10))
    cols, rows = n_elems[1], n_elems[0]
    for i in range(1, cols * rows + 1):
        sample_idx = torch.randint(len(data), size=(1,)).item()
        img, label = data[sample_idx]
        img = torch.permute(img, (1,2,0))
        figure.add_subplot(rows, cols, i)
        plt.title(labels_map[label])
        plt.axis('off')
        plt.imshow(img.squeeze())
    plt.show()